In [ ]:
import requests
import json
import pickle
import time
import csv

## [abo | gun | blm]
campaign = 'blm'

url_active_17 = 'http://130.56.248.5:9357/active2017/_search?pretty'

video_ids = []
with open('../data/social_media/{}/video_annotations.csv'.format(campaign)) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        video_id = row[0]
        label = row[3]
        if label == '1':
            video_ids.append(video_id)
        line_count += 1
    print('Processed {} lines.'.format(line_count))

print('len(video_ids):', len(video_ids))

videos = []
total = 0

for video_id in video_ids:
    scroll_ids = []

    query_json = {
        "size": 5,
        "query" : {
            "match_phrase" : {"id": video_id}
        }
    }

    query_str = json.dumps(query_json)

    headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

    r = requests.post(url_active_17, data=query_str, headers=headers)
    result = json.loads(r.content.decode('utf-8'))
    hits = result['hits']['hits']
    #print("# returned hits:", len(hits))
    num_total_hits = result['hits']['total']
    if num_total_hits != 1:
        print('There is a mistake for:', video_id)
        print("# total hits:", num_total_hits)
        print("returned hits: ", len(result['hits']['hits']))
    #print("max_score:", result['hits']['max_score'])
    total += len(hits)
    
    for hit in hits:
        videos.append(hit)
    
    print('total:', total)
    time.sleep(0.05)

print('#total returned videos:', len(videos))

results = {}
for video in videos:
    tweetId = video['_source']['id']
    results[tweetId] = video

print("# results: ", len(results.keys()))

try:
    pickle.dump(results, open("../data/social_media/{}/all_videos_by_relevant_videos.pkl".format(campaign), "wb"))
except:
    print("couldn't dump the videos!!!")
    pass
